# 🚀 LUMPS - Worker Distribuido para Google Colab

Este notebook permite ejecutar un worker distribuido para generar el dataset de la Fase 0 de LUMPS.

## 📋 Instrucciones

1. **Ejecuta las celdas en orden**
2. **Configura tu ruta de Google Drive** en la celda de configuración
3. **El worker trabajará automáticamente** hasta que se desconecte Colab
4. **Puedes pausar/reanudar** usando los botones de control

## ⚠️ Importante
- Cada usuario debe hacer **"Archivo > Guardar una copia en Drive"** antes de ejecutar
- El worker trabajará en **loop continuo** hasta desconexión
- Los resultados se guardan automáticamente en Google Drive compartido


## 🔧 Setup Inicial


In [ ]:
# Montar Google Drive
from google.colab import drive
import os

print("🔗 Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive montado exitosamente")


In [ ]:
# Clonar repositorio desde GitHub automáticamente
import subprocess
import sys
from pathlib import Path

print("📥 Clonando repositorio LUMPS...")

# Cambiar al directorio de trabajo
%cd /content

# URL del repositorio (actualizada automáticamente)
repo_url = "https://github.com/juanignaciorey/LUMPS.git"

# Verificar si ya existe el directorio
if Path("LUMPS").exists():
    print("📁 Repositorio ya existe, actualizando...")
    %cd LUMPS
    !git pull origin master
else:
    print("📥 Clonando repositorio...")
    !git clone $repo_url
    %cd LUMPS

print("✅ Repositorio listo")
print(f"📁 Directorio actual: {os.getcwd()}")


In [ ]:
# Instalar dependencias automáticamente
print("📦 Instalando dependencias...")

try:
    # Instalar requirements
    !pip install -r requirements.txt
    
    # Instalar dependencias adicionales para Colab
    !pip install tqdm
    
    print("✅ Dependencias instaladas exitosamente")
except Exception as e:
    print(f"⚠️ Error instalando dependencias: {e}")
    print("🔄 Intentando instalar dependencias básicas...")
    !pip install torch numpy tqdm
    print("✅ Dependencias básicas instaladas")


## ⚙️ Configuración


In [ ]:
# Crear carpeta compartida automáticamente si no existe
import os
from pathlib import Path

print("🔍 Verificando carpeta compartida...")

# Verificar si la carpeta base existe
if not Path(DRIVE_BASE_PATH).exists():
    print("📁 Creando carpeta compartida automáticamente...")
    try:
        Path(DRIVE_BASE_PATH).mkdir(parents=True, exist_ok=True)
        print(f"✅ Carpeta creada: {DRIVE_BASE_PATH}")
    except Exception as e:
        print(f"❌ Error creando carpeta: {e}")
        print("💡 Verifica que tienes permisos de escritura en Drive")
else:
    print(f"✅ Carpeta ya existe: {DRIVE_BASE_PATH}")

print("🎯 Verificación de carpeta completada")


## 🔧 Setup Automático

El notebook configurará automáticamente todo lo necesario:
- Creará la carpeta compartida si no existe
- Generará el manifiesto de batches automáticamente
- Configurará el worker para ejecutar


In [ ]:
# Setup automático - Crear manifiesto si no existe
import os
from pathlib import Path

print("🔍 Verificando setup inicial...")

# Verificar si ya existe el manifiesto
manifest_path = Path(DRIVE_BASE_PATH) / "manifests" / "batch_manifest.json"

if manifest_path.exists():
    print("✅ Manifiesto ya existe, continuando...")
    print(f"📁 Ubicación: {manifest_path}")
else:
    print("📋 Creando manifiesto de batches automáticamente...")
    print("⚠️ Esto puede tomar unos minutos para 500,000 tareas")

    try:
        # Crear manifiesto
        !python -m src.distributed.batch_generator \
            --drive-path $LUMPS_DRIVE_PATH \
            --create-manifest \
            --batch-size 50

        print("✅ Manifiesto creado exitosamente")
    except Exception as e:
        print(f"❌ Error creando manifiesto: {e}")
        print("💡 Asegúrate de que la carpeta LUMPS_Distributed existe en Drive")

print("🎯 Setup inicial completado")


In [ ]:
# Configurar paths y variables automáticamente
import os
import socket
import time
from pathlib import Path

# Ruta automática de la carpeta compartida
DRIVE_BASE_PATH = "/content/drive/MyDrive/LUMPS_Distributed"

# Generar ID único para este worker
WORKER_ID = f"colab_{socket.gethostname()}_{os.getpid()}_{int(time.time())}"

# Configurar variables de entorno
os.environ['LUMPS_DRIVE_PATH'] = DRIVE_BASE_PATH
os.environ['WORKER_ID'] = WORKER_ID

print(f"🔧 Configuración automática:")
print(f"   Worker ID: {WORKER_ID}")
print(f"   Drive Path: {DRIVE_BASE_PATH}")
print(f"   Hostname: {socket.gethostname()}")
print(f"   PID: {os.getpid()}")

# Verificar que la ruta existe (debería existir por las celdas anteriores)
if not Path(DRIVE_BASE_PATH).exists():
    print(f"⚠️ La ruta {DRIVE_BASE_PATH} no existe, creándola...")
    Path(DRIVE_BASE_PATH).mkdir(parents=True, exist_ok=True)
    print(f"✅ Ruta creada: {DRIVE_BASE_PATH}")
else:
    print(f"✅ Ruta verificada: {DRIVE_BASE_PATH}")

print("🎯 Configuración completada")


## 🖥️ Configuración de GPU


In [ ]:
# Verificar GPU disponible
import torch

print("🖥️ Verificando GPU...")
print(f"   CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("✅ GPU configurada y lista")
else:
    print("⚠️ No hay GPU disponible, usando CPU")

print(f"   PyTorch versión: {torch.__version__}")


## 🚀 Ejecutar Worker


In [ ]:
# Ejecutar worker distribuido automáticamente
print("🚀 Iniciando worker distribuido...")
print("⚠️ El worker trabajará en loop continuo hasta desconexión")
print("🛑 Presiona el botón de interrumpir para detener")
print("\n" + "="*80)

try:
    # Ejecutar worker con TQDM para progreso visual
    !python generate_dataset_distributed.py \
        --drive-path $LUMPS_DRIVE_PATH \
        --worker-id $WORKER_ID \
        --verbose
except Exception as e:
    print(f"❌ Error ejecutando worker: {e}")
    print("💡 Verifica que el manifiesto existe y tienes permisos de escritura")
    print("🔄 Intentando ejecutar con configuración básica...")
    !python generate_dataset_distributed.py \
        --drive-path $LUMPS_DRIVE_PATH \
        --worker-id $WORKER_ID
